# 1. 모듈, 패키지 불러오기

In [ ]:
import csv
import os
from pymongo import MongoClient
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from pathlib import Path

import time
import re
import datetime
from selenium.webdriver.support.select import Select

from tqdm import tqdm

# 2. 스크레이핑 대상 불러오기

- Section2 Project에서 수집하였던, 통신판매업 사업자등록번호를 불러옵니다.
- MongoDB 연결 함수를 작성합니다
- 팝업삭제 함수를 작성합니다

In [ ]:
# 사업자 등록번호 불러오기
target_biz_code = pd.read_csv("data/Section2_bizcode.csv")['사업자등록번호'].str.replace('-','')

In [ ]:
# DB 연결 
HOST = host
USER = user
PASSWORD = pwd
DATABASE_NAME = 'SMINFO'

def collection(name):
    COLLECTION_NAME = name
    MONGO_URI = f"mongodb+srv://{USER}:{PASSWORD}@{HOST}/{DATABASE_NAME}?retryWrites=true&w=majority"
    client = MongoClient({MONGO_URI})
    database = client[DATABASE_NAME]
    collection = database[COLLECTION_NAME]
        
    return collection

In [ ]:
# 팝업삭제
def del_popup(driver):
    main = driver.window_handles
    for handle in main: 
        if handle != main[0]: 
            driver.switch_to.window(handle) 
            driver.close()
            
    driver.switch_to.window(driver.window_handles[0])

# 3. 스크레이핑-1
## 중소기업 재무제표 불러오기

In [ ]:
log_id = id
log_pw = pw

def sminfo_crawling(log_id, log_pw):

    print("start crawling..")

    #접속
    chrome_options = Options() #브라우저 꺼짐 방지
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"]) 
    
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get('https://sminfo.mss.go.kr/cm/sv/CSV001R0.do')
    driver.implicitly_wait(10)
    del_popup(driver)

    login_id = driver.find_element(By.ID, 'login_id')
    login_id.clear()
    login_id.send_keys(log_id)

    login_pw = driver.find_element(By.ID, 'login_password')
    login_pw.clear()
    login_pw.send_keys(log_pw)
    login_bn = driver.find_element(By.CLASS_NAME, "login_btn").click()

    del_popup(driver)
    coll_bs = collection("balance_sheet")
    
    for search_code in tqdm(target_biz_code):
        try : searching(driver, search_code, coll_bs)
        except : continue
    

def searching(driver, search_code, coll_bs):
    
    driver.get('https://sminfo.mss.go.kr/gc/sf/GSF002R0.print')
    
    select = Select(driver.find_element(By.ID, "cmQueryOptionCombo"))
    select.select_by_visible_text("사업자번호")
    search_id = driver.find_element(By.ID, 'searchTxt')
    search_id.clear()
    search_id.send_keys(search_code)
    
    search_bn = driver.find_element(By.CLASS_NAME, value="btn_blue").click()
    
    try :
        if driver.find_element(By.CSS_SELECTOR, '#contents_sub > div:nth-child(3) > form:nth-child(5) > div > div.clearfix.mt20 > div.sm_text.fl > span').text == '0':
            return 
        else : 
            driver.find_element(By.XPATH, value='/html/body/div[1]/div[2]/div[6]/div/div[2]/div[3]/form[4]/div/div[7]/table/tbody/tr[1]/td[1]/a').click() 
            
    except : return
    
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.element_to_be_clickable((By.ID, 'btn_hbar')))
    table = pd.read_html(driver.page_source)[4]
    table['사업자등록번호'] = search_code
    coll_bs.insert_many(table.to_dict("records"))

In [ ]:
# 스크레이핑 함수 실행
sminfo_crawling(log_id, log_pw)

NameError: name 'sminfo_crawling' is not defined

In [ ]:
# MongoDB에 저장된 데이터를 드라이브에도 저장
pd.DataFrame(list(collection("balance_sheet").find({}))).to_csv("data/dataset_finacial.csv")

# 4. 스크레이핑-2
## 국세청 등록정보를 바탕으로 기업상태확인(폐업유무)

In [ ]:
# 재무제표를 불러올 수 있는 기업을 대상으로 사업자등록번호 추출
dataset_financial = pd.read_csv('data/dataset_finacial.csv')
df = dataset_financial['사업자등록번호'].drop_duplicates()
df

In [ ]:
def crawler(crawl_df, present_dataset):
    url =  'https://teht.hometax.go.kr/websquare/websquare.html?w2xPath=/ui/ab/a/a/UTEABAAA13.xml'
    
    driver = webdriver.Chrome('chromedriver')
    driver.implicitly_wait(5)
    driver.get(url)

    try :bsno_ = present_dataset['사업자등록번호'].tolist()
    except : bsno_ = []
    coll_biz_status = collection("biz_status")
    
    for bizno in tqdm(crawl_df.tolist()):

        if bizno in bsno_:
            continue

        else:
            bsno = driver.find_element(By.XPATH, '//*[@id="bsno"]')
            bsno.clear()
            bsno.send_keys(bizno)
            bsno.send_keys(Keys.RETURN)
            time.sleep(0.2)

            try : 
                coll_biz_status.insert_many(pd.read_html(driver.page_source)[0].to_dict("records"))
            except: continue
                
#데이터셋 불러오기
present_dataset = pd.DataFrame(list(collection("biz_status").find({})))
crawler(df, present_dataset)

In [ ]:
# MongoDB 저장 상태 확인
bsno_dataset = pd.DataFrame(list(collection("biz_status").find({})))
bsno_dataset

,_id,사업자등록번호,사업자등록상태,조회기준일자
0,63b1ba406aedbd1be90b15a8,101-81-00452,부가가치세 일반과세자 입니다.,2022-12-04
1,63b1ba406aedbd1be90b15a9,101-81-24696,부가가치세 일반과세자 입니다.,2022-12-04
2,63b1ba406aedbd1be90b15aa,101-81-62477,부가가치세 일반과세자 입니다.,2022-12-04
3,63b1ba406aedbd1be90b15ab,105-81-68075,부가가치세 일반과세자 입니다.,2022-12-04
4,63b1ba406aedbd1be90b15ac,128-86-79624,부가가치세 일반과세자 입니다.,2022-12-04
...,...,...,...,...
83304,63b1ba456aedbd1be90c5b10,767-81-01284,부가가치세 일반과세자 입니다.,2022-12-07
83305,63b1ba456aedbd1be90c5b11,873-86-01911,부가가치세 일반과세자 입니다.,2022-12-07
83306,63b1ba456aedbd1be90c5b12,108-81-83610,부가가치세 일반과세자 입니다.,2022-12-07
83307,63b1ba456aedbd1be90c5b13,120-81-01336,부가가치세 일반과세자 입니다.,2022-12-07


In [ ]:
# 전처리 적용하여 로컬 드라이브에 저장
bsno_dataset['폐업일자'] = bsno_dataset['사업자등록상태'].str.split('폐업일자:').str[1].str.extract(r'(\d+-\d+-\d+)')
bsno_dataset.to_csv("data/biz_status.csv")